In [1]:
import pandas as pd
df = pd.read_csv('car_data.csv')

/var/folders/bk/xhs0vwln2116v0d35hflv8b80000gn/T/ipykernel_72830/3352399789.py:2: DtypeWarning: Columns (80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('car_data.csv')


### Filling in null values

In [2]:
def fill_null_values(data, groupby_cols, result_col):
    return data[result_col].fillna(df.groupby(groupby_cols)[result_col].transform('mean')).round()

In [3]:
#### Horsepower
df['horsePower'] = fill_null_values(df, ['engineDescription'], 'horsePower')


#mpgCity
df['mpgCity'] = fill_null_values(df, ['make', 'model'], 'mpgCity')

df['mpgHighway'] = fill_null_values(df, ['make', 'model'], 'mpgHighway')

df['evMpgeCity'] = fill_null_values(df, ['make', 'model'], 'evMpgeCity')

df['evMpgeHighway'] = fill_null_values(df, ['make', 'model'], 'evMpgeHighway')

In [4]:
required_cols = [
    'make',
    'model',
    'bodyType',
    'year',
    'seating',
    'owners',
    'fuelDescription',
    'horsePower',
    'drivetrainDescription',
    'transmission',
    'mileage',
    'mpgCity',
    'mpgHighway',
    'evRange',
    'evMpgeCity',
    'evMpgeHighway',
    'isAutocheckReportAccidentVehicle',
    'price',
]

In [5]:
car_data = df[required_cols]

### Cleaning Transmission column

In [6]:
def cleanTransmission(x):

    if 'manual' in x.lower():
        return 'Manual'
    else:
        return 'Auto'
    

car_data.loc[:, 'transmission']=car_data['transmission'].apply(lambda x: cleanTransmission(x))
car_data.loc[:,'transmission'].value_counts()

transmission
Auto      17911
Manual     1486
Name: count, dtype: int64

## Cleaning Drive Train Column

In [7]:
def cleanDriveTrain(x):

    if 'awd' in x.lower():
        return 'AWD'
    else:
        return x
    

car_data.loc[:, 'drivetrainDescription']=car_data['drivetrainDescription'].apply(lambda x: cleanDriveTrain(x))
car_data.loc[:,'drivetrainDescription'].value_counts()

drivetrainDescription
FWD    7655
AWD    6069
4WD    2990
RWD    2550
2WD     133
Name: count, dtype: int64

### Finding Age of the car from Year

In [8]:
car_data['age'] = 2025 - car_data['year']

/var/folders/bk/xhs0vwln2116v0d35hflv8b80000gn/T/ipykernel_72830/675059998.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car_data['age'] = 2025 - car_data['year']


### Changing Column names

In [9]:
car_data.rename(columns={
    'seating': 'seats', 
    'owners': 'numOfOwners',
    'fuelDescription': 'fuelType',
    'drivetrainDescription': 'driveTrain',
    'milage': 'milesDriven',
    'isAutocheckReportAccidentVehicle': 'hasReportedAccident'}, inplace=True)

/var/folders/bk/xhs0vwln2116v0d35hflv8b80000gn/T/ipykernel_72830/3713363305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car_data.rename(columns={


In [10]:
car_data.drop(columns=['year'], inplace=True)

/var/folders/bk/xhs0vwln2116v0d35hflv8b80000gn/T/ipykernel_72830/2212989868.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car_data.drop(columns=['year'], inplace=True)


In [11]:
mask = car_data['fuelType'] == 'Electric'
car_data.loc[mask, ['mpgCity', 'mpgHighway']] = car_data.loc[mask, ['mpgCity', 'mpgHighway']].fillna(0)

In [12]:
mask = car_data['fuelType'] == 'Gas'
car_data.loc[mask, ['evRange', 'evMpgeCity', 'evMpgeHighway']] = car_data.loc[mask, ['evRange', 'evMpgeCity', 'evMpgeHighway']].fillna(0)

In [13]:
mask = car_data['fuelType'] == 'Flexible Fuel'
car_data.loc[mask, ['evRange', 'evMpgeCity', 'evMpgeHighway']] = car_data.loc[mask, ['evRange', 'evMpgeCity', 'evMpgeHighway']].fillna(0)

In [20]:
len(car_data)

18485

In [15]:
len(car_data.dropna())

18485

In [16]:
car_data['bodyType'].unique().tolist()

['Wagon',
 'Truck',
 'Sedan',
 'SUV',
 'Convertible',
 'Hatchback',
 'Van',
 'Coupe',
 'Minivan']

In [67]:
car_data.columns

Index(['make', 'model', 'bodyType', 'seats', 'numOfOwners', 'fuelType',
       'horsePower', 'driveTrain', 'transmission', 'mileage', 'mpgCity',
       'mpgHighway', 'evRange', 'evMpgeCity', 'evMpgeHighway',
       'hasReportedAccident', 'price', 'age'],
      dtype='object')

In [17]:
car_data['hasReportedAccident'].value_counts()

hasReportedAccident
False    16528
True      2869
Name: count, dtype: int64

In [18]:
car_data = car_data.dropna()

car_data.to_csv('cleaned_data.csv', index=False)

In [93]:
filter_df = car_data[['make', 'model', 'bodyType', 'fuelType','driveTrain', 'transmission', 'seats', 'horsePower']]

In [94]:
filter_df = filter_df.drop_duplicates()

In [95]:
filter_df.to_csv('filter_data.csv', index=False)

In [82]:
car_data['fuelType'].unique()

array(['Gas', 'Electric', 'Hybrid', 'Flexible Fuel', 'Diesel',
       'Plug-In Hybrid'], dtype=object)

In [19]:
car_data[car_data['fuelType']=='Electric'].isnull().sum()

make                   0
model                  0
bodyType               0
seats                  0
numOfOwners            0
fuelType               0
horsePower             0
driveTrain             0
transmission           0
mileage                0
mpgCity                0
mpgHighway             0
evRange                0
evMpgeCity             0
evMpgeHighway          0
hasReportedAccident    0
price                  0
age                    0
dtype: int64

In [90]:
car_data[car_data['fuelType']=='Electric']

,make,model,bodyType,seats,numOfOwners,fuelType,horsePower,driveTrain,transmission,mileage,mpgCity,mpgHighway,evRange,evMpgeCity,evMpgeHighway,hasReportedAccident,price,age
9,Tesla,Model 3,Sedan,5,1,Electric,510.0,AWD,Auto,4472,0.0,0.0,315.0,118.0,107.0,False,32990,4
11,Volkswagen,ID.4,SUV,5,2,Electric,201.0,RWD,Auto,24226,0.0,0.0,250.0,104.0,89.0,False,25990,3
36,Audi,Q4 e-tron,SUV,5,1,Electric,201.0,RWD,Auto,12462,0.0,0.0,265.0,112.0,94.0,False,39590,1
61,Rivian,R1S,SUV,7,1,Electric,600.0,AWD,Auto,19772,0.0,0.0,321.0,82.0,74.0,False,77990,2
91,Tesla,Model 3,Sedan,5,1,Electric,510.0,AWD,Auto,35879,0.0,0.0,385.0,134.0,126.0,False,28990,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14835,Kia,EV6,SUV,5,1,Electric,320.0,AWD,Auto,27349,0.0,0.0,274.0,116.0,94.0,False,29990,3
14840,Tesla,Model S,Sedan,5,1,Electric,362.0,RWD,Auto,37343,0.0,0.0,265.0,88.0,90.0,False,24590,10
14852,Tesla,Model Y,SUV,5,2,Electric,442.0,AWD,Auto,21679,0.0,0.0,303.0,115.0,106.0,False,29990,4
14888,Chevrolet,Bolt EV,Hatchback,5,1,Electric,200.0,FWD,Auto,41307,0.0,0.0,259.0,131.0,109.0,False,18990,2
